In [ ]:
import numpy as np
from binomial import Binomial
from calibration import calibration
from rcn import rcn
import statsmodels.formula.api as smf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
"""Run callibration file and return the interest rate, div yield, u and d"""
r, y, u, d = calibration()

## Question 1

In [ ]:
#Sepcify other parameters

T = 12
dt = 1/T
i0 = 11118
c = 0.1
alpha = 1
beta = 0.8

In [ ]:
#prints the price of the bond with given params using both implementations
tree = Binomial(r, T, dt, i0, u, d, y)
#Replicating portfolio price
print('Price of the notes with given parameters are:')
print('Replicating Portfolio\n')
print('{:10} : {:.4f}'.format(' rcn', tree.price_RCN(alpha, c)))
print('{:10} : {:.4f}'.format('brcn', tree.price_RCN(alpha, c, beta)))
#print('{:10} : {:.4f}'.format('bond', tree.price_bond(c*dt)))

#Direct calculation price
note = rcn(r, dt, i0, y, u, d, c, T)
print('\nDirect Calculation')
p_rcn, rep_rcn = note.price_rcn(alpha=alpha, c=c, replication_strategy=True)
p_brcn, rep_brcn = note.price_brcn(alpha=alpha, beta=beta, c=c, replication_strategy=True)
print('{:10} : {:.4f}'.format(' rcn', p_rcn))
print('{:10} : {:.4f}'.format('brcn', p_brcn))
print('\nReplicating Strategy of RCN:\nRiskless={:.6f}\nRisky={:.6f}'.format(rep_rcn[1], rep_rcn[0]))
print('\nReplicating Strategy of BRCN:\nRiskless={:.6f}\nRisky={:.6f}'.format(rep_brcn[1], rep_brcn[0]))
#print('{:10} : {:.4f}'.format('bond', note.bond))


## Question 2

In [ ]:
par_price = 1

def f(c, *args): #specify the function to be minimized
    alpha, beta, type = args
    if type == 'RCN': p = tree.price_RCN(alpha, c)
    if type == 'BRCN': p = tree.price_RCN(alpha, c, beta)
    return (par_price - p) ** 2

res_RCN = minimize(f, 0, args=(alpha, beta, 'RCN'))
res_BRCN = minimize(f, 0, args=(alpha, beta, 'BRCN'))

print('alpha = {}, beta = {}'.format(alpha, beta))

if res_RCN.success: print('Par coupon for RCN:  c = {:.2%}'.format(res_RCN.x[0]))
if res_BRCN.success: print('Par coupon for BRCN: c = {:.2%}'.format(res_BRCN.x[0]))

In [ ]:
res_RCN = minimize(f, 0, args=(alpha, beta, 'RCN'))
rates = [0.4,0.6,0.8,1]
alphas = np.linspace(0.6, 1, 10)
betas = np.outer(alphas, rates)

cons = ({'type': 'ineq', 'fun': lambda x: 1 - x},   #constrains ensuring that par-alpha is within the [0, 1]
        {'type': 'ineq', 'fun': lambda x: x - 0})

rcn_c = []
brcn_c = np.zeros_like(betas)

for i, b in enumerate(tqdm(betas)): #runs loop for different rates and strikes
    alpha = alphas[i]
    res_RCN = minimize(f, 0, args=(alpha, beta, 'RCN'), constraints=cons)
    rcn_c.append(res_RCN.x[0]*100)
    for j, beta in enumerate(b):
        res_BRCN = minimize(f, 0, args=(alpha, beta, 'BRCN'), constraints=cons)
        brcn_c[i,j] = res_BRCN.x[0]*100

In [ ]:
plt.plot(alphas, rcn_c)
for c in brcn_c.T:
    plt.plot(alphas, c)

r08_a = brcn_c.T[-2]
r1_a = brcn_c.T[-1]

plt.xlabel(r'$\alpha$')
plt.ylabel('Par Coupon Rate (%)')
plt.legend(['RCN'] + [r'beta = {}$\alpha$'.format(r) for r in rates])
plt.title(r'Par-coupon rate as a function of $\alpha$')
plt.show()

In [ ]:
coupons = np.linspace(0, 0.09, 10)
par_price = 1


cons = ({'type': 'ineq', 'fun': lambda x: 1 - x}, #constraints on alpha
        {'type': 'ineq', 'fun': lambda x: x - 0})

def g(alpha, *args):
    c, = args
    return (par_price - tree.price_RCN(alpha, c)) ** 2

alphas = []
for c in coupons:
    res = minimize(g, 1, args=(c), constraints=cons)
    alphas.append(res.x[0])

plt.plot(coupons*100, alphas)
plt.xlabel('Coupon Rate (%)')
plt.ylabel(r'Par-$\alpha$')
plt.title(r'Par-$\alpha$ as a function of coupon rate for basic RCN')
plt.show(tree.price_RCN(alpha, c))

In [ ]:
par_price = 1

def h(alpha, *args):
    c, rate = args
    return (par_price - tree.price_RCN(alpha, c, rate*alpha)) ** 2

alphas = np.linspace(0.6, 1, 20)
coupons = np.linspace(0, 0.09, 10)
rates = [0.4, 0.6, 0.8, 1]

par_alph = np.zeros([len(rates), coupons.shape[0]])
for n, r in enumerate(rates):
    dist = np.zeros([alphas.shape[0], coupons.shape[0]])
    for j, c in enumerate(tqdm(coupons)):
        for i, a in enumerate(alphas):
            dist[i, j] = h(a, c, r)
    min_alpha_ind = np.argmin(dist, axis=0)
    par_alph[n, :] = alphas[min_alpha_ind]

In [ ]:
cols = ['orange', 'green', 'red', 'purple']
r08_c, r1_c = [], []
for i, r in enumerate(rates):
    if r == 0.8: r08_c = par_alph[i]
    if r == 1.0: r1_c = par_alph[i]
    plt.plot(coupons*100,  par_alph[i], label=r'$\beta$={}$\alpha$'.format(r), color=cols[i])
    plt.xlabel('Coupon Rate (%)')
    plt.ylabel(r'Par $\alpha$')

plt.title(r'Par-$\alpha$ as a function of coupon rate for barrier RCN')
plt.legend()
plt.show()


#Q6-7-8

In [ ]:
#dates = [i for i in range(int(T/dt)+1)]
alpha = 1
c = 0.1
beta = 0.8
dates = [j for j in range(1, T)]
RCN, rep1 = note.price_rcn(alpha=alpha, c=c, dates=dates, replication_strategy=True)
BRCN, rep2 = note.price_brcn(alpha=alpha, c=c, beta=0.8, dates=dates, replication_strategy=True)
print('Price of callable simple RCN {:.4f}\nPrice of callable barrier RCN {:.4f}'.format(RCN, BRCN))
p_rcn, rep_rcn = note.price_rcn(alpha=alpha, c=c, replication_strategy=True)

print('\nReplicating Strategy of callable RCN:\nRiskless={:.6f}\nRisky={:.6f}'.format(rep1[1], rep1[0]))
print('\nReplicating Strategy of calabe BRCN:\nRiskless={:.6f}\nRisky={:.6f}'.format(rep2[1], rep2[0]))

In [ ]:
par_price = 1

def f(c, *args):
    alpha, beta, type, dates = args
    if type == 'RCN': p = note.price_rcn(alpha, c, dates)
    if type == 'BRCN': p = note.price_brcn(alpha, beta, c, dates)
    return (par_price - p) ** 2

cons = ({'type': 'ineq', 'fun': lambda x: 1 - x},
        {'type': 'ineq', 'fun': lambda x: x })


rates = [0.4,0.6,0.8,1]
alphas = np.linspace(0.6, 1, 10)
betas = np.outer(alphas, rates)

rcn_c = []
brcn_c = np.zeros_like(betas)

for i, b in enumerate(tqdm(betas)):
    alpha = alphas[i]
    res_RCN = minimize(f, 0, args=(alpha, beta, 'RCN', dates), constraints=cons)
    rcn_c.append(res_RCN.x[0]*100)
    for j, beta in enumerate(b):
        res_BRCN = minimize(f, 0, args=(alpha, beta, 'BRCN', dates), constraints=cons)
        brcn_c[i,j] = res_BRCN.x[0]*100

In [ ]:

alphas = np.linspace(0.6, 1, 10)
plt.plot(alphas, rcn_c)
for c in brcn_c.T:
    plt.plot(alphas, c)
plt.plot(alphas, r08_a)
plt.plot(alphas, r1_a)
plt.xlabel(r'$\alpha$')
plt.ylabel('Par Coupon rate (%)')
plt.legend(['RCN'] + [r'beta = {}$\alpha$'.format(r) for r in rates] + [r'(NC) $\beta$ = 0.8$\alpha$'] + [r'(NC) $\beta$ = 1$\alpha$'])
plt.title(r'Par-coupon rate as a function of $\alpha$ (callable)')
plt.savefig('5_7.pdf')
plt.show()

Grid Search

In [ ]:
#Grid search for finding par-alpha

coupons = np.linspace(0., 0.12, 15)
alphas = np.linspace(0.6, 1, 100)
rates = [0.4, 0.6, 0.8, 1]

par_alph = np.zeros([len(rates), coupons.shape[0]])

for n, r in enumerate(rates):
    dist = np.zeros([alphas.shape[0], coupons.shape[0]])
    for j, c in enumerate(tqdm(coupons)):
        for i, a in enumerate(alphas):
            p = note.price_brcn(a, r*a, c, dates)
            dist[i, j] = (1 - p)**2
    min_alpha_ind = np.argmin(dist, axis=0)
    par_alph[n, :] = alphas[min_alpha_ind]

In [ ]:

alphas = np.linspace(0.6, 1, 100)
for i, r in enumerate(reversed(rates)):
    plt.plot(coupons*100,  par_alph[i], label=r'$\beta$={}$\alpha$'.format(r))
    plt.xlabel('Coupon Rate (%)')
    plt.ylabel(r'Par $\alpha$')

plt.plot(np.linspace(0., 0.12, 10)*100, r08_c)
plt.plot(np.linspace(0., 0.12, 10)*100, r1_c)
plt.legend([r'$\beta$={}$\alpha$'.format(r) for r in rates] + [r'(NC) $\beta$ = 0.8$\alpha$'] + [r'(NC) $\beta$ = 1$\alpha$'])
plt.title(r'Par-$\alpha$ as a function of coupon rate for barrier RCN (callable)')
plt.savefig('5_8.pdf')
plt.show()